In [ ]:
!pip install sentence_transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats as st
import seaborn as sns
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 5.3 MB 24.2 MB/s 
     |████████████████████████████████| 1.3 MB 49.9 MB/s 
     |████████████████████████████████| 163 kB 55.9 MB/s 
     |████████████████████████████████| 7.6 MB 21.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=544c2321535c23f9dceea80eea07a93a6330af4bbffb5c16f788e7039202a1b0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

### Чтение 

In [ ]:
pd.set_option('max_colwidth', 500)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
products = pd.read_excel('https://docs.google.com/spreadsheets/d/1qrDeOleXax7zHNcIFJz0hu3q_kXjfcY6/export?format=xlsx', header=1)

In [ ]:
industries = pd.read_excel('https://docs.google.com/spreadsheets/d/1Y0ze45q-4M_cPqtgQttru-DFgoL5TOxS/export?format=xlsx')

In [ ]:
technologies = pd.read_excel('https://docs.google.com/spreadsheets/d/1SQ-cYrNULnZ8Rfiaog9p_EWf5CgIpYlC/export?format=xlsx')

### Исследование

In [ ]:
def general_information(data):
    data.info()
    print()
    display(data.head(20))
    print()
    print('Дубликатов', data.duplicated().sum(), 'или', round(data.duplicated().sum() / len(data) * 100), '%')

In [ ]:
general_information(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6894 entries, 0 to 6893
Data columns (total 12 columns):
 #   Column                                                    Non-Null Count  Dtype 
---  ------                                                    --------------  ----- 
 0   global_id (продукта)                                      6894 non-null   int64 
 1   Компания                                                  6894 non-null   int64 
 2   Рыночное наименование продукта                            6894 non-null   object
 3   Раздел (ОКПД 2)                                           6894 non-null   object
 4   Класс (ОКПД 2)                                            6894 non-null   object
 5   Подкласс (ОКПД 2)                                         6894 non-null   object
 6   Группа (ОКПД 2)                                           6894 non-null   object
 7   Подгруппа (ОКПД 2)                                        6894 non-null   object
 8   Вид (ОКПД 2)                

,global_id (продукта),Компания,Рыночное наименование продукта,Раздел (ОКПД 2),Класс (ОКПД 2),Подкласс (ОКПД 2),Группа (ОКПД 2),Подгруппа (ОКПД 2),Вид (ОКПД 2),Категория (ОКПД 2),Подкатегория (ОКПД 2),Описание продукта (с указанием конкретных характеристик)
0,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц..."
1,812474,811423,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.2. Услуги, связанные с научными исследованиями и экспериментальными разработками в области технических наук и в области технологий, кроме биотехнологии","72.19.21. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий","72.19.21.000. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий",Отсутствует,"Проект предполагает создание методик и фреймворка для автоматизации построения корпоративных хранилищ данных. Библиотеки фреймворка предоставляют готовые инструменты для решения задач с данными.Основным преимуществом компании «Дататех» является наличие уникальной экспертизы построения и управления хранилищами данных с использованием современных Open SoUrce технологий, экспертизы в решениях по созданию и управлению жизненным циклом данных, позволяющих повысить качество и прозрачность данных, ..."
2,812477,811310,StreamMyData,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью",73. Услуги рекламные и услуги по исследованию конъюнктуры рынка,73.1. Услуги рекламные,"73.11. Услуги, предоставляемые рекламными агентствами","73.11.1. Услуги, предоставляемые рекламными агентствами",73.11.11. Набор рекламных услуг полный,73.11.11.000. Набор рекламных услуг полный,Отсутствует,"Цифровая онлайн-платформа «StreamMyData», которая предназначена для оптимизации рекламного бюджета компании и автоматизации функций маркетолога.Преимущества компании «MediaNation» следующие. 1) Специалисты регулярно проходят сертификацию в «Яндекс» и «google» и подтверждают свою квалификацию работы с инструментами для анализа аудитории, такими как «Яндекс.Метрика» и «google AnalyticS». Это означает, что компания сможет профессионально проводить исследование тематики в онлайне.2) При проведен..."
3,812480,811499,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.1. Услуги, с


Дубликатов 0 или 0 %


### Замена типов

In [ ]:
def type_change(data, column, type): # функция для замены типов данных 
    if type == 'datetime':
        data[column] = pd.to_datetime(data[column])
        print(data[column].dtypes)
    else:
        data[column] = data[column].astype(type, errors='ignore')
        print(data[column].dtypes)

### Преобразование названий столбцов

In [ ]:
products.columns = products.columns.str.rstrip()  # убираем пробел в конце строки

In [ ]:
products.columns = products.columns.str.replace('(', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
products.columns = products.columns.str.replace(')', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
products.columns = products.columns.str.replace(' ', '_')

In [ ]:
technologies.columns = technologies.columns.str.rstrip()  # убираем пробел в конце строки

In [ ]:
technologies.columns = technologies.columns.str.replace('(', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
technologies.columns = technologies.columns.str.replace(')', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
technologies.columns = technologies.columns.str.replace(' ', '_')

In [ ]:
industries.columns = industries.columns.str.replace(' ', '_')

In [ ]:
general_information(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6894 entries, 0 to 6893
Data columns (total 12 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   global_id_продукта                                      6894 non-null   int64 
 1   Компания                                                6894 non-null   int64 
 2   Рыночное_наименование_продукта                          6894 non-null   object
 3   Раздел_ОКПД_2                                           6894 non-null   object
 4   Класс_ОКПД_2                                            6894 non-null   object
 5   Подкласс_ОКПД_2                                         6894 non-null   object
 6   Группа_ОКПД_2                                           6894 non-null   object
 7   Подгруппа_ОКПД_2                                        6894 non-null   object
 8   Вид_ОКПД_2                                      

,global_id_продукта,Компания,Рыночное_наименование_продукта,Раздел_ОКПД_2,Класс_ОКПД_2,Подкласс_ОКПД_2,Группа_ОКПД_2,Подгруппа_ОКПД_2,Вид_ОКПД_2,Категория_ОКПД_2,Подкатегория_ОКПД_2,Описание_продукта_с_указанием_конкретных_характеристик
0,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц..."
1,812474,811423,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.2. Услуги, связанные с научными исследованиями и экспериментальными разработками в области технических наук и в области технологий, кроме биотехнологии","72.19.21. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий","72.19.21.000. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий",Отсутствует,"Проект предполагает создание методик и фреймворка для автоматизации построения корпоративных хранилищ данных. Библиотеки фреймворка предоставляют готовые инструменты для решения задач с данными.Основным преимуществом компании «Дататех» является наличие уникальной экспертизы построения и управления хранилищами данных с использованием современных Open SoUrce технологий, экспертизы в решениях по созданию и управлению жизненным циклом данных, позволяющих повысить качество и прозрачность данных, ..."
2,812477,811310,StreamMyData,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью",73. Услуги рекламные и услуги по исследованию конъюнктуры рынка,73.1. Услуги рекламные,"73.11. Услуги, предоставляемые рекламными агентствами","73.11.1. Услуги, предоставляемые рекламными агентствами",73.11.11. Набор рекламных услуг полный,73.11.11.000. Набор рекламных услуг полный,Отсутствует,"Цифровая онлайн-платформа «StreamMyData», которая предназначена для оптимизации рекламного бюджета компании и автоматизации функций маркетолога.Преимущества компании «MediaNation» следующие. 1) Специалисты регулярно проходят сертификацию в «Яндекс» и «google» и подтверждают свою квалификацию работы с инструментами для анализа аудитории, такими как «Яндекс.Метрика» и «google AnalyticS». Это означает, что компания сможет профессионально проводить исследование тематики в онлайне.2) При проведен..."
3,812480,811499,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.1. Услуги, связанные с научными 


Дубликатов 0 или 0 %


### Замена пустых значений

In [ ]:
products.loc[products['Подкатегория_ОКПД_2'].isna(),'Подкатегория_ОКПД_2'] = 'Отсутствует'

In [ ]:
products.loc[products['Подкатегория_ОКПД_2'].isna(),'Категория_ОКПД_2'] = 'Отсутствует'

In [ ]:
products.loc[products['Вид_ОКПД_2'].isna(),'Вид_ОКПД_2'] = 'Отсутствует'

In [ ]:
general_information(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6894 entries, 0 to 6893
Data columns (total 12 columns):
 #   Column                                                  Non-Null Count  Dtype 
---  ------                                                  --------------  ----- 
 0   global_id_продукта                                      6894 non-null   int64 
 1   Компания                                                6894 non-null   int64 
 2   Рыночное_наименование_продукта                          6894 non-null   object
 3   Раздел_ОКПД_2                                           6894 non-null   object
 4   Класс_ОКПД_2                                            6894 non-null   object
 5   Подкласс_ОКПД_2                                         6894 non-null   object
 6   Группа_ОКПД_2                                           6894 non-null   object
 7   Подгруппа_ОКПД_2                                        6894 non-null   object
 8   Вид_ОКПД_2                                      

,global_id_продукта,Компания,Рыночное_наименование_продукта,Раздел_ОКПД_2,Класс_ОКПД_2,Подкласс_ОКПД_2,Группа_ОКПД_2,Подгруппа_ОКПД_2,Вид_ОКПД_2,Категория_ОКПД_2,Подкатегория_ОКПД_2,Описание_продукта_с_указанием_конкретных_характеристик
0,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц..."
1,812474,811423,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.2. Услуги, связанные с научными исследованиями и экспериментальными разработками в области технических наук и в области технологий, кроме биотехнологии","72.19.21. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий","72.19.21.000. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий",Отсутствует,"Проект предполагает создание методик и фреймворка для автоматизации построения корпоративных хранилищ данных. Библиотеки фреймворка предоставляют готовые инструменты для решения задач с данными.Основным преимуществом компании «Дататех» является наличие уникальной экспертизы построения и управления хранилищами данных с использованием современных Open SoUrce технологий, экспертизы в решениях по созданию и управлению жизненным циклом данных, позволяющих повысить качество и прозрачность данных, ..."
2,812477,811310,StreamMyData,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью",73. Услуги рекламные и услуги по исследованию конъюнктуры рынка,73.1. Услуги рекламные,"73.11. Услуги, предоставляемые рекламными агентствами","73.11.1. Услуги, предоставляемые рекламными агентствами",73.11.11. Набор рекламных услуг полный,73.11.11.000. Набор рекламных услуг полный,Отсутствует,"Цифровая онлайн-платформа «StreamMyData», которая предназначена для оптимизации рекламного бюджета компании и автоматизации функций маркетолога.Преимущества компании «MediaNation» следующие. 1) Специалисты регулярно проходят сертификацию в «Яндекс» и «google» и подтверждают свою квалификацию работы с инструментами для анализа аудитории, такими как «Яндекс.Метрика» и «google AnalyticS». Это означает, что компания сможет профессионально проводить исследование тематики в онлайне.2) При проведен..."
3,812480,811499,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.1. Услуги, связанные с научными 


Дубликатов 0 или 0 %


### Соединение таблиц

In [ ]:
# соеденим products с industries, с условием, что global_id (продукта) и Создаваемые продукты совпадают
products_and_industries = products.merge(industries, how='left', left_on='global_id_продукта', right_on='Создаваемые_продукты')

In [ ]:
# соеденим products и technologies, с условием, что global_id (продукта) и Создаваемые продукты совпадают
products_and_technologies = products.merge(technologies, how='left', left_on='global_id_продукта', right_on='Создаваемые_продукты')

In [ ]:
products_and_industries = products_and_industries.drop(['Создаваемые_продукты'], axis=1) 
products_and_technologies = products_and_technologies.drop(['Создаваемые_продукты'], axis=1) 

In [ ]:
products_and_industries = products_and_industries.drop_duplicates(keep = 'first')
products_and_technologies = products_and_technologies.drop_duplicates(keep = 'first')

In [ ]:
products_and_industries.loc[products_and_industries['Подотрасль'].isna(),'Подотрасль'] = 'Отсутствует'
products_and_technologies.loc[products_and_technologies['Технология_3_уровень'].isna(),'Технология_3_уровень'] = 'Отсутствует'

In [ ]:
general_information(products_and_industries)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13737 entries, 0 to 13736
Data columns (total 15 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   global_id_продукта                                      13737 non-null  int64  
 1   Компания                                                13737 non-null  int64  
 2   Рыночное_наименование_продукта                          13737 non-null  object 
 3   Раздел_ОКПД_2                                           13737 non-null  object 
 4   Класс_ОКПД_2                                            13737 non-null  object 
 5   Подкласс_ОКПД_2                                         13737 non-null  object 
 6   Группа_ОКПД_2                                           13737 non-null  object 
 7   Подгруппа_ОКПД_2                                        13737 non-null  object 
 8   Вид_ОКПД_2                          

,global_id_продукта,Компания,Рыночное_наименование_продукта,Раздел_ОКПД_2,Класс_ОКПД_2,Подкласс_ОКПД_2,Группа_ОКПД_2,Подгруппа_ОКПД_2,Вид_ОКПД_2,Категория_ОКПД_2,Подкатегория_ОКПД_2,Описание_продукта_с_указанием_конкретных_характеристик,global_id,Отрасль,Подотрасль
0,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц...",830847.0,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,Добыча нефти и природного газа
1,812474,811423,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.2. Услуги, связанные с научными исследованиями и экспериментальными разработками в области технических наук и в области технологий, кроме биотехнологии","72.19.21. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий","72.19.21.000. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий",Отсутствует,"Проект предполагает создание методик и фреймворка для автоматизации построения корпоративных хранилищ данных. Библиотеки фреймворка предоставляют готовые инструменты для решения задач с данными.Основным преимуществом компании «Дататех» является наличие уникальной экспертизы построения и управления хранилищами данных с использованием современных Open SoUrce технологий, экспертизы в решениях по созданию и управлению жизненным циклом данных, позволяющих повысить качество и прозрачность данных, ...",831547.0,ФИНАНСОВЫЙ СЕКТОР,Банковская деятельность
2,812474,811423,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.2. Услуги, связанные с научными исследованиями и экспериментальными разработками в области технических наук и в области технологий, кроме биотехнологии","72.19.21. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий","72.19.21.000. Услуги, связанные с научными исследованиями и экспериментальными разработками в области нанотехнологий",Отсутствует,"Проект предполагает создание методик и фреймворка для автоматизации построения корпоративных хранилищ данных. Библиотеки фреймворка предоставляют готовые инструменты для решения задач с данными.Основным преимуществом компании «Дататех» является наличие уникальной экспертизы построения и управления хранилищами данных с использованием современных Open SoUrce технологий, экспертизы в решениях по созданию и 


Дубликатов 0 или 0 %


In [ ]:
general_information(products_and_technologies)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8165 entries, 0 to 8164
Data columns (total 16 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   global_id_продукта                                      8165 non-null   int64  
 1   Компания                                                8165 non-null   int64  
 2   Рыночное_наименование_продукта                          8165 non-null   object 
 3   Раздел_ОКПД_2                                           8165 non-null   object 
 4   Класс_ОКПД_2                                            8165 non-null   object 
 5   Подкласс_ОКПД_2                                         8165 non-null   object 
 6   Группа_ОКПД_2                                           8165 non-null   object 
 7   Подгруппа_ОКПД_2                                        8165 non-null   object 
 8   Вид_ОКПД_2                            

,global_id_продукта,Компания,Рыночное_наименование_продукта,Раздел_ОКПД_2,Класс_ОКПД_2,Подкласс_ОКПД_2,Группа_ОКПД_2,Подгруппа_ОКПД_2,Вид_ОКПД_2,Категория_ОКПД_2,Подкатегория_ОКПД_2,Описание_продукта_с_указанием_конкретных_характеристик,global_id,Технология_1_уровень,Технология_2_уровень,Технология_3_уровень
0,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц...",833824.0,Синтез информации,Моделирование процессов реального мира (цифровые двойники),Моделирование физических процессов
1,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц...",833843.0,Синтез информации,Моделирование процессов реального мира (цифровые двойники),Моделирование инженерных систем
2,812463,811059,-,Раздел B. Продукция горнодобывающих производств,09. Услуги в области добычи полезных ископаемых,09.9. Услуги в горнодобывающем производстве прочие,09.90. Услуги в горнодобывающем производстве прочие,09.90.1. Услуги в горнодобывающем производстве прочие,"09.90.19. Услуги в горнодобывающем производстве, не включенные в другие группировки",09.90.19.100. Услуги по добыче и первичному обогащению урановых и ториевых руд,Отсутствует,"Конечным продуктом проекта является роторная управляемая система -высокоскоростная гидроимпульсная телеметрия, интегрированная с комплексной геофизической аппаратурой сканирующего типа диаметрами 120 и 172 мм.Забойная телеметрическая система с интегральными геофизическими измерительными модулями и гидроимпульсным каналом передачи данных позволяет обеспечить технологические превосходства по сравнению с зарубежными аналогами. Оборудование разрабатывается в рамках программы импортозамещения с ц...",853808.0,Синтез материальных объектов,Новые производственные процессы,Процессы и технологии добычи полезных ископаемых
3,812474,811423,-,"Раздел M. Услуги, связанные с научной, инженерно-технической и профессиональной деятельностью","72. Услуги и работы, связанные с научными исследованиями и экспериментальными разработками","72.1. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук","72.19. Услуги, связанные с научными исследованиями и экспериментальными разработками в области естественных и технических наук, прочие","72.19.2. 


Дубликатов 0 или 0 %


# Поиск несоответсвий

In [ ]:
def compare(desc, cat):
    # Two lists of sentences
    if desc == '' or cat == '':
      return False
    sentences = desc.split('.')

    category = [cat] * len(sentences)

    #Compute embedding for both lists
    embeddings1 = model.encode(sentences, convert_to_tensor=True)
    embeddings2 = model.encode(category, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)

    #Output the pairs with their score
    for i in range(len(sentences)):
        if (cosine_scores[i][i] >= .3):
            return True
    return False


In [ ]:
productsOKPD = products.copy()
type_change(productsOKPD, 'Описание_продукта_с_указанием_конкретных_характеристик', 'string')
type_change(products_and_industries, 'Описание_продукта_с_указанием_конкретных_характеристик', 'string')
type_change(products_and_technologies, 'Описание_продукта_с_указанием_конкретных_характеристик', 'string')
type_change(productsOKPD, 'Вид_ОКПД_2', 'string')
type_change(productsOKPD, 'Категория_ОКПД_2', 'string')
type_change(productsOKPD, 'Подкатегория_ОКПД_2', 'string')
type_change(products_and_industries, 'Подотрасль', 'string')
type_change(products_and_technologies, 'Технология_3_уровень', 'string')


string
string
string
string
string
string
string
string


In [ ]:
print(len(productsOKPD))
print(len(products_and_industries))
print(len(products_and_technologies))

6894
13737
8165


In [ ]:
for i in range(len(productsOKPD)):
  if (productsOKPD['Подкатегория_ОКПД_2'][i] != "Отсутствует"):
    productsOKPD.at[i ,'Сравнение_ОКПД_2'] = compare(productsOKPD['Описание_продукта_с_указанием_конкретных_характеристик'][i], productsOKPD['Подкатегория_ОКПД_2'][i])
  elif(productsOKPD['Категория_ОКПД_2'][i] != "Отсутствует"):
    productsOKPD.at[i ,'Сравнение_ОКПД_2'] = compare(productsOKPD['Описание_продукта_с_указанием_конкретных_характеристик'][i], productsOKPD['Категория_ОКПД_2'][i])
  elif(productsOKPD['Вид_ОКПД_2'][i] != "Отсутствует"):
    productsOKPD.at[i ,'Сравнение_ОКПД_2'] = compare(productsOKPD['Описание_продукта_с_указанием_конкретных_характеристик'][i], productsOKPD['Вид_ОКПД_2'][i])
if (products_and_industries['Подотрасль'][i] != "Отсутствует"):
  for i in range(len(products_and_industries)):
    products_and_industries.at[i ,'Сравнение_с_подотрослью'] = compare(products_and_industries['Описание_продукта_с_указанием_конкретных_характеристик'][i], products_and_industries['Подотрасль'][i])
if (products_and_industries['Технология_3_уровень'][i] != "Отсутствует"):
  for i in range(100):
    products_and_technologies.at[i ,'Сравнение_с_технологией_3_уровеня'] = compare(products_and_technologies['Описание_продукта_с_указанием_конкретных_характеристик'][i], products_and_technologies['Технология_3_уровень'][i])


KeyboardInterrupt: ignored

In [ ]:
productsOKPD.groupby('Сравнение_ОКПД_2')[['Сравнение_ОКПД_2']].count()

In [ ]:
products_and_industries.groupby('Сравнение_с_подотрослью')[['Сравнение_с_подотрослью']].count()

In [ ]:
products_and_technologies.groupby('Сравнение_с_технологией_3_уровеня')[['Сравнение_с_технологией_3_уровеня']].count()

In [ ]:
productsOKPDFalse = productsOKPD.copy()
productsOKPDFalse = productsOKPDFalse.query('Сравнение_ОКПД_2 == False')

In [ ]:
products_and_industriesFalse = products_and_industries.copy()
products_and_industriesFalse = products_and_industriesFalse.query('Сравнение_с_подотрослью == False')

In [ ]:
products_and_technologiesFalse = products_and_technologies.copy()
products_and_technologiesFalse = products_and_technologiesFalse.query('Сравнение_с_технологией_3_уровеня == False')

In [ ]:
general_information(productsOKPDFalse)

In [ ]:
productsOKPD.to_excel('ProductsOKPD.xlsx', index=False)

In [ ]:
productsOKPDFalse.to_excel('FailedproductsOKPD.xlsx', index=False)

In [ ]:
products_and_industries.to_excel('products_and_industries.xlsx', index=False)

In [ ]:
products_and_industriesFalse.to_excel('Failedproducts_and_industries.xlsx', index=False)

In [ ]:
products_and_technologies.to_excel('products_and_technologies.xlsx', index=False)

In [ ]:
products_and_technologiesFalse.to_excel('Failedproducts_and_technologies.xlsx', index=False)